In [1]:
from calibration_kabsch import cal_transformation_kabsch, Transformation
import numpy as np
import pyrealsense2 as rs
import dt_apriltags
import cv2

In [ ]:
# Ros
import rospy
import message_filters
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

class Image_Converter(object):
    
    def __init__(self, color_topic, depth_topic=None):
        self.bridge = CvBridge()

        image_sub = message_filters.Subscriber(color_topic, Image)
        depth_sub = message_filters.Subscriber(depth_topic, Image)

        self.timeSync = message_filters.ApproximateTimeSynchronizer(
                        [image_sub, depth_sub], 10, 0.5)
        self.timeSync.registerCallback(self.callback)
        
    def callback(self, image_msg, depth_msg):
        header = image_msg.header
        secs = header.stamp.secs
        nsecs = header.stamp.nsecs
        try:
            image_data = self.bridge.imgmsg_to_cv2(image_msg, "bgr8")
            depth_data = self.bridge.imgmsg_to_cv2(depth_msg, "passthrough")
        except CvBridgeError as e:
            print(e)
            
        depth_data = np.array(depth_data, dtype=np.uint16)


In [ ]:
def post_process_depth_frame(depth_frame, decimation_magnitude=1.0, spatial_magnitude=2.0, spatial_smooth_alpha=0.5,
                             spatial_smooth_delta=20, temporal_smooth_alpha=0.4, temporal_smooth_delta=20):
    """
    Filter the depth frame acquired using the Intel RealSense device

    Parameters:
    -----------
    depth_frame          : rs.frame()
                           The depth frame to be post-processed
    decimation_magnitude : double
                           The magnitude of the decimation filter
    spatial_magnitude    : double
                           The magnitude of the spatial filter
    spatial_smooth_alpha : double
                           The alpha value for spatial filter based smoothening
    spatial_smooth_delta : double
                           The delta value for spatial filter based smoothening
    temporal_smooth_alpha: double
                           The alpha value for temporal filter based smoothening
    temporal_smooth_delta: double
                           The delta value for temporal filter based smoothening

    Return:
    ----------
    filtered_frame : rs.frame()
                     The post-processed depth frame
    """

    # Post processing possible only on the depth_frame
    assert (depth_frame.is_depth_frame())

    # Available filters and control options for the filters
    decimation_filter = rs.decimation_filter()
    spatial_filter = rs.spatial_filter()
    temporal_filter = rs.temporal_filter()

    filter_magnitude = rs.option.filter_magnitude
    filter_smooth_alpha = rs.option.filter_smooth_alpha
    filter_smooth_delta = rs.option.filter_smooth_delta

    # Apply the control parameters for the filter
    decimation_filter.set_option(filter_magnitude, decimation_magnitude)
    spatial_filter.set_option(filter_magnitude, spatial_magnitude)
    spatial_filter.set_option(filter_smooth_alpha, spatial_smooth_alpha)
    spatial_filter.set_option(filter_smooth_delta, spatial_smooth_delta)
    temporal_filter.set_option(filter_smooth_alpha, temporal_smooth_alpha)
    temporal_filter.set_option(filter_smooth_delta, temporal_smooth_delta)

    # Apply the filters
    filtered_frame = decimation_filter.process(depth_frame)
    filtered_frame = spatial_filter.process(filtered_frame)
    filtered_frame = temporal_filter.process(filtered_frame)

    return filtered_frame

In [9]:
class RSCalibration:
    
    def __init__(self, intrinsic=None, tag_family='tagStandard41h12', tag_size=None):

        self.at_detector = dt_apriltags.Detector(searchpath=['apriltags'],
                               families=tag_family,
                               nthreads=1,
                               quad_decimate=1.0,
                               quad_sigma=0.0,
                               refine_edges=1,
                               decode_sharpening=0.25,
                               debug=0)
        self.intrinsic = intrinsic
        self.tag_size = tag_size
        
    def convert_depth_pixel_to_metric_coordinate(self, depth, pixel_x, pixel_y):
        X = (pixel_x - self.intrinsic.ppx) / self.intrinsic.fx * depth
        Y = (pixel_y - self.intrinsic.ppy) / self.intrinsic.fy * depth
        return X, Y, depth
        
    def get_3d_corners(self, img, depth):
        '''
        Args:
        ------
        img: (H, W, C) ndarray, 1 for gray or 3 for color.
        depth: (H, W) ndarray.
        '''
        assert img.shape[:2] == depth.shape[:2]
        if img.shape[-1] == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif img.shape[-1] == 1:
            gray = img
        else:
            raise Exception('Only support gray or color.')
        detections = self.at_detector.detect(gray)
        if detections:
            corners2D = {}
            for each in detections:
                corners, center = each.corners.tolist(), each.center.tolist()
                corners.append(center)
                corners2D[each.tag_id] = corners
                
            corners3D = {}
            for k, v in corners2D.items():
                corners3D[k] = []
                for point in v:
                    X = int(round(point[0]))
                    Y = int(round(point[1]))
                    Z = 3.1 # depth[Y, X]
                    if Z == 0 or Z is None:
                        Z = None
                    # Convert
                    else:
                        X, Y, Z = self.convert_depth_pixel_to_metric_coordinate(Z, X, Y)
                    corners3D[k].append([X, Y, Z])
            return corners3D
        else:
            return



def perform_RT(src3D, dst3D):
    
    srcPoints = []
    dstPoints = []
    
    for k in src3D.keys():
        if not k in dst3D.keys():
            continue
        for i in range(len(src3D[k])):
            src_point = src3D[k][i]
            dst_point = dst3D[k][i]
            # Check Z value is ok.
            
            if src_point[2] and dst_point[2]:
                srcPoints.append(src_point)
                dstPoints.append(dst_point)
    if len(srcPoints) < 5:
        return None, None

    srcPoints = np.asarray(srcPoints)
    dstPoints = np.asarray(dstPoints)
    R, T, rmsd_value = cal_transformation_kabsch(srcPoints, dstPoints)
    print(rmsd_value)
    return R, T

In [10]:
RS15 = RSCalibration(intrinsic=rs15_camera_intrinsics)
RS16 = RSCalibration(intrinsic=rs16_camera_intrinsics)

In [ ]:
at_detector = dt_apriltags.Detector(searchpath=['apriltags'],
                               families='tagStandard41h12',
                               nthreads=1,
                               quad_decimate=1.0,
                               quad_sigma=0.0,
                               refine_edges=1,
                               decode_sharpening=0.25,
                               debug=0)

gray = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)
import matplotlib.pyplot as plt
plt.imshow(gray)
detections = at_detector.detect(gray)

In [ ]:
detections[0].corners

In [ ]:
detections[0].center

In [ ]:
points = [[604.20355225, 320.29168701],
       [587.84625244, 319.8927002 ],
       [587.66259766, 335.98678589],
       [604.03594971, 336.80117798],
       [595.83307789, 328.23411996]]
for point in points:
    x = round(point[0])
    y = round(point[1])
    cv2.circle(gray, (x,y), 1,(0,0,0), 0)
cv2.imshow('gray', gray)
k = cv2.waitKey()
if k == 27:
    cv2.destroyAllWindows()

In [11]:
data_dir = '/home/commaai-03/Data/color/apriltags/test'
color = '/home/commaai-03/Data/color/apriltags/rs15_0.jpg'
depth = '/home/commaai-03/Data/color/apriltags/rs15_0.png'

color = cv2.imread(color)
depth = cv2.imread(depth, cv2.IMREAD_ANYDEPTH) / 1000.

corners3D1 = RS15.get_3d_corners(color, depth)

In [12]:
data_dir = '/home/commaai-03/Data/color/apriltags/test'
color = '/home/commaai-03/Data/color/apriltags/rs16_0.jpg'
depth = '/home/commaai-03/Data/color/apriltags/rs16_0.png'

color = cv2.imread(color)
depth = cv2.imread(depth, cv2.IMREAD_ANYDEPTH) / 1000.

corners3D2 = RS16.get_3d_corners(color, depth) 
corners3D2

{9: [[1.074002480766813, 0.46997833622140717, 3.1],
  [0.9883162115302764, 0.46997833622140717, 3.1],
  [0.9933565803088962, 0.5556567150420272, 3.1],
  [1.074002480766813, 0.5556567150420272, 3.1],
  [1.0336795305378546, 0.5153374779499708, 3.1]],
 11: [[0.8018225667213438, 0.4800581454944213, 3.1],
  [0.7211766662634271, 0.4800581454944213, 3.1],
  [0.7211766662634271, 0.5606966196785342, 3.1],
  [0.8068629354999637, 0.5606966196785342, 3.1],
  [0.7665399852710053, 0.5203773825864777, 3.1]],
 21: [[1.0689621119881931, 0.1827037719405048, 3.1],
  [0.9883162115302764, 0.1827037719405048, 3.1],
  [0.9883162115302764, 0.26334224612461776, 3.1],
  [1.074002480766813, 0.2683821507611248, 3.1],
  [1.0286391617592348, 0.22302300903256125, 3.1]],
 99: [[0.8018225667213438, 0.17766386730399775, 3.1],
  [0.7211766662634271, 0.17262396266749067, 3.1],
  [0.7211766662634271, 0.2583023414881107, 3.1],
  [0.8018225667213438, 0.2583023414881107, 3.1],
  [0.7614996164923855, 0.2179831043960542, 3.1]]

In [13]:
corners3D1

{9: [[1.339393940605284, -0.3822363663571299, 3.1],
  [1.2480298032901305, -0.3822363663571299, 3.1],
  [1.2480298032901305, -0.29598367432195777, 3.1],
  [1.3343181551988865, -0.29598367432195777, 3.1],
  [1.2937118719477072, -0.3416468642229313, 3.1]],
 11: [[1.060225743253426, -0.3822363663571299, 3.1],
  [0.9739373913446698, -0.3873100541239047, 3.1],
  [0.9688616059382724, -0.3010573620887326, 3.1],
  [1.0551499578470287, -0.29598367432195777, 3.1],
  [1.0145436745958492, -0.3416468642229313, 3.1]],
 21: [[1.3546212968244762, -0.6714365690632951, 3.1],
  [1.2632571595093227, -0.67651025683007, 3.1],
  [1.2632571595093227, -0.5902575647948979, 3.1],
  [1.3495455114180788, -0.5902575647948979, 3.1],
  [1.3089392281668994, -0.6308470669290965, 3.1]],
 99: [[1.0754530994726181, -0.6917313201303945, 3.1],
  [0.989164747563862, -0.6968050078971693, 3.1],
  [0.989164747563862, -0.6105523158619972, 3.1],
  [1.0754530994726181, -0.6054786280952225, 3.1],
  [1.0297710308150414, -0.651141817

In [ ]:
clip = depth[273:350, 572:642]
clip.max()

In [ ]:
b = depth > 0
c = depth * b
d = c.ravel()[np.flatnonzero(c)]
np.sort(d)

In [14]:
perform_RT(corners3D1, corners3D2)

0.00695752950406751


(array([[ 9.98724374e-01,  5.04938024e-02,  1.84182523e-29],
        [-5.04938024e-02,  9.98724374e-01, -1.79300218e-27],
        [-1.08930255e-28,  1.78978497e-27,  1.00000000e+00]]),
 array([-0.23863015,  0.91777755,  0.        ]))

In [2]:
from easydict import EasyDict

rs15_camera_intrinsics = EasyDict()
rs15_camera_intrinsics.ppx = 424.120849609375
rs15_camera_intrinsics.ppy = 241.3369903564453
rs15_camera_intrinsics.fx = 610.742919921875
rs15_camera_intrinsics.fy = 610.9954223632812

rs16_camera_intrinsics = EasyDict()
rs16_camera_intrinsics.ppx = 428.91986083984375
rs16_camera_intrinsics.ppy = 238.74856567382812
rs16_camera_intrinsics.fx = 615.0343627929688
rs16_camera_intrinsics.fy = 615.0910034179688

In [ ]:
# store image

import os
import numpy as np
import cv2
import time
import rospy
import message_filters
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image


class SaveImage(object):
    
    def __init__(self):
        self.rsidlist = ['rs15', 'rs16', 'rs17', 'rs18']
        self.save_dir = os.path.join('/home/ai-cap02/workspace/data',
                                     'apriltags')
        rospy.init_node('save_images', anonymous=True)
    
    def callback(self, color, depth, rsid):
        header = color.header
        secs = header.stamp.secs
        nsecs = header.stamp.nsecs 
        color = CvBridge().imgmsg_to_cv2(color, desired_encoding=color.encoding)
        depth = CvBridge().imgmsg_to_cv2(depth, desired_encoding=depth.encoding)
        
        path_color = os.path.join(self.save_dir, rsid, 'color')
        path_depth = os.path.join(self.save_dir, rsid, 'depth')
        if not os.path.exists(path_color):
            os.makedirs(path_color)
        if not os.path.exists(path_depth):
            os.makedirs(path_depth)
        
        file_color = os.path.join(path_color, '%s.%s.jpg' %(secs, nsecs))
        file_depth = os.path.join(path_depth, '%s.%s.png' %(secs, nsecs))
        
        cv2.imwrite(file_color, color)
        cv2.imwrite(file_depth, depth)
        print('[Info]: %s saved!' % file_color)
        

    def run(self):
        for rsid in self.rsidlist:
            color_sub = message_filters.Subscriber('/' + rsid + '/color/image_rect_color', Image)
            depth_sub = message_filters.Subscriber('/' + rsid + '/aligned_depth_to_color/image_raw', Image)
            ts = message_filters.ApproximateTimeSynchronizer([color_sub, depth_sub], 10, 0.02)
            ts.registerCallback(self.callback, rsid)
            
        rospy.spin()

In [46]:
import cv2

def showImg(img):
    cv2.imshow('showImg', img)
    k = cv2.waitKey()
    if k:
        cv2.destroyAllWindows()
        

        
img_file = '/home/commaai-03/Data/color/test/rs15_0.jpg'
img = cv2.imread(img_file, 0)

h = [[(i, 0),(i, 480)] for i in range(0, 848, 13)]
v = [[(0, i), (848, i)] for i in range(0, 480, 7)]

loop = len(h) if len(h) < len(v) else len(v)

for i in range(loop):
    cv2.line(img, h[i][0], h[i][1], (0, 0, 255), 1, 1)
    cv2.line(img, v[i][0], v[i][1], (0, 0, 255), 1, 1)

showImg(img)

In [42]:
h = [[(i, 0),(i, 480)] for i in range(0, 848, 13)]
v = [[(0, i), (848, i)] for i in range(0, 480, 7)]

In [43]:
len(v)

69

In [44]:
h

[[(0, 0), (0, 480)],
 [(13, 0), (13, 480)],
 [(26, 0), (26, 480)],
 [(39, 0), (39, 480)],
 [(52, 0), (52, 480)],
 [(65, 0), (65, 480)],
 [(78, 0), (78, 480)],
 [(91, 0), (91, 480)],
 [(104, 0), (104, 480)],
 [(117, 0), (117, 480)],
 [(130, 0), (130, 480)],
 [(143, 0), (143, 480)],
 [(156, 0), (156, 480)],
 [(169, 0), (169, 480)],
 [(182, 0), (182, 480)],
 [(195, 0), (195, 480)],
 [(208, 0), (208, 480)],
 [(221, 0), (221, 480)],
 [(234, 0), (234, 480)],
 [(247, 0), (247, 480)],
 [(260, 0), (260, 480)],
 [(273, 0), (273, 480)],
 [(286, 0), (286, 480)],
 [(299, 0), (299, 480)],
 [(312, 0), (312, 480)],
 [(325, 0), (325, 480)],
 [(338, 0), (338, 480)],
 [(351, 0), (351, 480)],
 [(364, 0), (364, 480)],
 [(377, 0), (377, 480)],
 [(390, 0), (390, 480)],
 [(403, 0), (403, 480)],
 [(416, 0), (416, 480)],
 [(429, 0), (429, 480)],
 [(442, 0), (442, 480)],
 [(455, 0), (455, 480)],
 [(468, 0), (468, 480)],
 [(481, 0), (481, 480)],
 [(494, 0), (494, 480)],
 [(507, 0), (507, 480)],
 [(520, 0), (520, 